# Testing the Algorithm 
To test if the algorithm works properly and in alignment with the instructions from the Project Guidelines, **5 tests**will be run. These include:  
- **2 exception tests**: what does the algorithm do, if unexpected inputs are passed to it?  
- **2 functional tests**: does the algorithm output correct solutions if adequate inputs are provided? This especially pertains to the amount of transactions (are they really minimized) and the path that it outputs (are the correct amounts transferred to and from the correct people). Functional tests will be verified through comparison with the actual Splitwise-app. 

In [7]:
from splitwise import OptimalSplit
import pytest


## Exception Tests


## Functional Tests

## Performance Test
Does the algorithm solve transactions with more than 15 people in less than 1 second? 

In [4]:
transactions = [
    ['Person11', 'Person2', 4],
    ['Person12', 'Person5', 32],
    ['Person4',  'Person3', 95],
    ['Person2',  'Person11',95],
    ['Person15', 'Person9', 12],
    ['Person10', 'Person7', 5],
    ['Person1',  'Person2', 28],
    ['Person4',  'Person9', 78],
    ['Person1',  'Person9', 26],
    ['Person12', 'Person11',90],
    ['Person9',  'Person7', 29],
    ['Person8',  'Person10',36],
    ['Person13', 'Person14',1],
    ['Person13', 'Person15',21],
    ['Person12', 'Person7', 44],
]


splitter = OptimalSplit()
txns = splitter.minTransfers(transactions)

print(f"{len(txns)} payments need to be made.")

for debtor, creditor, amount in txns:
    print(f"{debtor} pays {creditor} {amount}")

defaultdict(<class 'int'>, {'Person11': -181, 'Person2': 63, 'Person12': 166, 'Person5': -32, 'Person4': 173, 'Person3': -95, 'Person15': -9, 'Person9': -87, 'Person10': -31, 'Person7': -78, 'Person1': 54, 'Person8': 36, 'Person13': 22, 'Person14': -1})
{'Person11': -181, 'Person2': 63, 'Person12': 166, 'Person5': -32, 'Person4': 173, 'Person3': -95, 'Person15': -9, 'Person9': -87, 'Person10': -31, 'Person7': -78, 'Person1': 54, 'Person8': 36, 'Person13': 22, 'Person14': -1}


**Answer**: No. The algoritm needs to be optimized accordingly. This is a to-do. 

#### Quick and Dirty Fix
Needs to be implemented properly at later stage. 

In [5]:
from collections import defaultdict
from math import inf, isclose
from functools import lru_cache
import math

class test:
    """
    Core algorithm for debt splitting with minimal amount of payments,
    with memoization and lower‑bound pruning for efficiency.
    """
    def test_transfers(self, transactions):
        """
        Performs debt split with minimal number of payments.

        Args:
        - transactions: list of [lender, borrower, amount]

        Returns:
        - A list of [creditor, debtor, amount] transactions.
        """
        # 1. Compute net scores
        score = defaultdict(int)
        for lender, borrower, amount in transactions:
            score[lender]   += amount
            score[borrower] -= amount

        # 2. Filter out zero balances
        debt = {p: bal for p, bal in score.items() if not isclose(bal, 0, abs_tol=1e-9)}
        people   = list(debt.keys())
        balances = tuple(debt[p] for p in people)

        @lru_cache(maxsize=None)
        def search(state):
            """
            state: tuple of balances aligned with `people`
            returns: (min_payments, path_tuple)
            """
            b = list(state)
            n = len(b)

            # Skip settled
            idx = 0
            while idx < n and isclose(b[idx], 0, abs_tol=1e-9):
                idx += 1
            # All settled
            if idx == n:
                return 0, ()

            # Lower‑bound pruning: at least ceil(unsettled/2) payments needed
            unsettled = [x for x in b if not isclose(x, 0, abs_tol=1e-9)]
            lb = math.ceil(len(unsettled) / 2)

            best_count = inf
            best_path  = ()

            # Try pairing idx with any later opposite‑sign balance
            for j in range(idx + 1, n):
                if b[idx] * b[j] < 0:
                    amt = min(abs(b[idx]), abs(b[j]))

                    # Apply transaction
                    saved_i = b[idx]
                    saved_j = b[j]
                    if b[idx] < 0:
                        b[idx] += amt
                        b[j]   -= amt
                        step = (people[idx], people[j], round(amt, 2))
                    else:
                        b[idx] -= amt
                        b[j]   += amt
                        step = (people[j], people[idx], round(amt, 2))

                    # Recurse
                    next_state = tuple(b)
                    cnt, subpath = search(next_state)
                    total = cnt + 1
                    if total < best_count:
                        best_count = total
                        best_path  = (step,) + subpath

                    # Backtrack
                    b[idx] = saved_i
                    b[j]   = saved_j

                    # Prune if we've hit the lower bound
                    if best_count == lb:
                        break

            return best_count, best_path

        # Get result and return as list of lists
        _, path = search(balances)
        return [list(txn) for txn in path]




transactions = [
    ['Person11', 'Person2', 4],
    ['Person12', 'Person5', 32],
    ['Person4',  'Person3', 95],
    ['Person2',  'Person11',95],
    ['Person15', 'Person9', 12],
    ['Person10', 'Person7', 5],
    ['Person1',  'Person2', 28],
    ['Person4',  'Person9', 78],
    ['Person1',  'Person9', 26],
    ['Person12', 'Person11',90],
    ['Person9',  'Person7', 29],
    ['Person8',  'Person10',36],
    ['Person13', 'Person14',1],
    ['Person13', 'Person15',21],
    ['Person12', 'Person7', 44],
]

splitter = test()
trans = splitter.test_transfers(transactions)

print(f"{len(trans)} payments need to be made.")

for debtor, creditor, amount in trans:
    print(f"{debtor} pays {creditor} {amount}")


11 payments need to be made.
Person11 pays Person12 166
Person11 pays Person1 15
Person5 pays Person2 32
Person10 pays Person2 31
Person3 pays Person4 95
Person7 pays Person4 78
Person15 pays Person1 9
Person9 pays Person1 30
Person9 pays Person8 36
Person9 pays Person13 21
Person14 pays Person13 1
